### Dataset explorations -- full customer service records


In [2]:
import duckdb
import pandas as pd
import os.path

home = os.path.expanduser("~")

file_path = os.path.join(home, "misc", "seattle-all-fixit-requests.csv")
conn = duckdb.connect()

# Get the first 10 rows of the CSV file
result = conn.execute(f"SELECT * FROM read_csv('{file_path}') LIMIT 10")

# Print the result -- do not wrap columns
pd.set_option('display.expand_frame_repr', False)
print(result.fetchdf())

  Service Request Number                        Service Request Type                            City Department        Created Date         Method Received              Status                                   Location       X_Value        Y_Value   Latitude   Longitude                 Latitude/Longitude  ZIP Code  Council District Police Precinct     Neighborhood
0            21-00000006  Street Sign and Traffic Signal Maintenance  SDOT-Seattle Department of Transportation 2021-01-01 05:24:02     Find It Fix It Apps            Reported      2052 RAINIER AVE S, SEATTLE, WA 98144  1.277820e+06  216740.084565  47.584693 -122.302683  POINT (-122.30268292 47.58469328)     98144                 3           SOUTH         ATLANTIC
1            21-00000007                                    Graffiti               SPU-Seattle Public Utilities 2021-01-01 07:31:00     Find It Fix It Apps            Reported       2920 DEXTER AVE N, SEATTLE, WA 98109  1.266760e+06  239220.678480  47.645724 -122.34

In [5]:
# Create a set() of all possible `Status` values
result = conn.execute(f"SELECT DISTINCT Status FROM read_csv('{file_path}')")
statuses = set(result.fetchdf()["Status"].tolist())
for status in sorted(list(statuses)):
	print(status)


Canceled
Closed
Closed -Incomplete Information
Closed as Duplicate
Duplicate (Closed)
Duplicate (Open)
New
Open
Reported
Transferred to Other Dept


In [6]:
# Create a set() of all `Service Request Type` values
result = conn.execute(f"SELECT DISTINCT \"Service Request Type\" FROM read_csv('{file_path}')")
service_request_types = set(result.fetchdf()["Service Request Type"].tolist())
for service_request_type in sorted(list(service_request_types)):
	print(service_request_type)

ADA Request (Transportation)
Abandoned Vehicle
Abandoned Vehicle/72hr Parking Ordinance
Animal Noise
Bicycle Facility Maintenance
Business Related Complaint
Business Violation of Public Health Requirements
Clogged Storm Drain
Damaged Sidewalk
Dead Animal
Feedback about the Customer Service Requests Portal
Feedback about the Find It, Fix It mobile app
Found a Pet
General Inquiry - Animal Shelter
General Inquiry - City Light
General Inquiry - Customer Service Bureau
General Inquiry - Police Department
General Inquiry - Public Utilities
General Inquiry - Transportation
Graffiti
Illegal Dumping / Needles
Internet/Cable Issue
Lost a Pet
Nightlife Noise Complaint
Nuisance dogs in a park
Overgrown Vegetation
Parking Enforcement
Parks and Recreation Maintenance
Pollution Report Form
Pothole
Public Garage or Parking Lot Complaint
Public Litter and Recycling Cans
Public Utilities Direct Mail Opt Out
Safe Routes to School
Scooter or Bike Share Issue
Seattle Center
Snow and Ice
Street Sign Mainten